In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

from utils import load_dataset, flatten_data, select_emotions, train_test_split
from utils import Autoencoder

/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
data, labels, groups = flatten_data(data, labels, groups)
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)
for em1 in range(4):
    for em2 in range(em1+1, 4):
        accs = []
        for human_test in range(15):
            e_data, e_labels, e_groups = select_emotions(data, labels, groups, em1, em2)
            train_data, train_labels, test_data, test_labels = train_test_split(e_data, e_labels, e_groups, human_test)

            xgboost = XGBClassifier()
            xgboost.fit(train_data, train_labels)
            pred = xgboost.predict(test_data)
            acc = accuracy_score(test_labels, pred)
            accs.append(acc)
        print(f"Emotion {em1} vs {em2}: {np.mean(accs)}")




Emotion 0 vs 1: 0.5135449735449735
Emotion 0 vs 2: 0.5143253968253968
Emotion 0 vs 3: 0.5226587301587301
Emotion 1 vs 2: 0.5139814814814815
Emotion 1 vs 3: 0.5307275132275132
Emotion 2 vs 3: 0.5310317460317461


In [ ]:
data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
data, labels, groups = flatten_data(data, labels, groups)
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)
for em1 in range(4):
    for em2 in range(em1+1, 4):
        accs = []
        for human_test in range(15):
            e_data, e_labels, e_groups = select_emotions(data, labels, groups, em1, em2)
            train_data, train_labels, test_data, test_labels = train_test_split(e_data, e_labels, e_groups, human_test)

            knn = KNeighborsClassifier(n_neighbors=10)
            knn.fit(train_data, train_labels)
            pred = knn.predict(test_data)
            acc = accuracy_score(test_labels, pred)
            accs.append(acc)
        print(f"Emotion {em1} vs {em2}: {np.mean(accs)}")


In [ ]:
data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
data, labels, groups = flatten_data(data, labels, groups)
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

pca = PCA(n_components=10)
pca.fit(data)
data = pca.transform(data)

for em1 in range(4):
    for em2 in range(em1+1, 4):
        accs = []
        for human_test in range(15):
            e_data, e_labels, e_groups = select_emotions(data, labels, groups, em1, em2)
            train_data, train_labels, test_data, test_labels = train_test_split(e_data, e_labels, e_groups, human_test)

            knn = KNeighborsClassifier(n_neighbors=10)
            knn.fit(train_data, train_labels)
            pred = knn.predict(test_data)
            acc = accuracy_score(test_labels, pred)
            accs.append(acc)
        print(f"Emotion {em1} vs {em2}: {np.mean(accs)}")


Emotion 0 vs 1: 0.5132407407407408
Emotion 0 vs 2: 0.5143518518518518
Emotion 0 vs 3: 0.5098809523809523
Emotion 1 vs 2: 0.5104629629629629
Emotion 1 vs 3: 0.5152777777777777
Emotion 2 vs 3: 0.505925925925926


In [27]:
data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
data, labels, groups = flatten_data(data, labels, groups)
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

data, labels, groups = select_emotions(data, labels, groups, 0, 2)

train_data, train_labels, test_data, test_labels = train_test_split(data, labels, groups, 0)

print(data.shape)

data_tensor = torch.tensor(train_data, dtype=torch.float32)
data_labels = torch.tensor(train_labels, dtype=torch.float32)[..., None]
batch_size = 1024
dataset = TensorDataset(data_tensor, data_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)[..., None]

# Initialize the autoencoder


(5400, 2480)


In [6]:
def objective(trial):
    batch_size = trial.suggest_int('batch_size', 128, 2048)
    lr = trial.suggest_float('lr', 1e-6, 1e-2, log=True)
    input_dim = data.shape[1]
    encoding_dim = 128  # Set the desired encoding dimension
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    autoencoder = Autoencoder(input_dim, encoding_dim)

    # Initialize weights of the autoencoder
    def init_weights(m):
        if type(m) == torch.nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    autoencoder.apply(init_weights)

    # Loss function and optimizer
    criterion_enc = torch.nn.MSELoss()
    criterion_cls = torch.nn.BCELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

    # Training loop
    num_epochs = 50
    best_acc = 0
    for epoch in range(num_epochs):
        for data_batch, labels_batch in dataloader:
            reconstructed, cls = autoencoder(data_batch)
            loss1 = criterion_cls(cls, labels_batch) * 0.1
            loss2 = criterion_enc(reconstructed, data_batch) 
            loss = loss1 + loss2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        with torch.no_grad():
            reconstructed, cls = autoencoder(data_tensor)
            cls = cls > 0.5
            train_acc = torch.sum(cls == data_labels) / data_labels.shape[0]
            # test acc

            reconstructed, cls = autoencoder(test_data_tensor)
            cls = cls > 0.5
            test_acc = torch.sum(cls == test_labels) / test_labels.shape[0]
            if epoch > 15 and test_acc < 0.5:
                break
            if test_acc > best_acc:
                best_acc = test_acc
                print(f"New best acc: {best_acc} epoch {epoch}")
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Acc: {train_acc.item()}, Test Acc: {test_acc.item()}')
    return best_acc



In [7]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
print(study.best_params)
# {'batch_size': 578, 'lr': 0.0004021431860941115}

[I 2024-06-04 00:37:09,772] A new study created in memory with name: no-name-5361f203-13bd-47e1-8e26-fc05af1ad764


New best acc: 0.5123015642166138 epoch 0
Epoch [1/50], Loss: 0.8480, Train Acc: 0.5333333611488342, Test Acc: 0.5123015642166138
Epoch [2/50], Loss: 0.8454, Train Acc: 0.49166667461395264, Test Acc: 0.5021825432777405
Epoch [3/50], Loss: 0.8340, Train Acc: 0.5444444417953491, Test Acc: 0.5051587224006653
Epoch [4/50], Loss: 0.8302, Train Acc: 0.5083333253860474, Test Acc: 0.4968253970146179
Epoch [5/50], Loss: 0.8224, Train Acc: 0.5222222208976746, Test Acc: 0.5019841194152832
Epoch [6/50], Loss: 0.8277, Train Acc: 0.4972222149372101, Test Acc: 0.49861112236976624
New best acc: 0.5174603462219238 epoch 6
Epoch [7/50], Loss: 0.8191, Train Acc: 0.5222222208976746, Test Acc: 0.5174603462219238
Epoch [8/50], Loss: 0.8142, Train Acc: 0.5416666865348816, Test Acc: 0.509325385093689
Epoch [9/50], Loss: 0.8099, Train Acc: 0.519444465637207, Test Acc: 0.5134920477867126
Epoch [10/50], Loss: 0.8016, Train Acc: 0.5055555701255798, Test Acc: 0.5067460536956787
Epoch [11/50], Loss: 0.8056, Train Ac

[I 2024-06-04 00:37:13,148] Trial 0 finished with value: 0.5204365253448486 and parameters: {'batch_size': 901, 'lr': 6.302275618557322e-05}. Best is trial 0 with value: 0.5204365253448486.


Epoch [16/50], Loss: 0.7750, Train Acc: 0.5416666865348816, Test Acc: 0.5152778029441833
New best acc: 0.5017856955528259 epoch 0
Epoch [1/50], Loss: 0.8442, Train Acc: 0.5083333253860474, Test Acc: 0.5017856955528259
New best acc: 0.5063492059707642 epoch 1
Epoch [2/50], Loss: 0.7707, Train Acc: 0.5222222208976746, Test Acc: 0.5063492059707642
Epoch [3/50], Loss: 0.7359, Train Acc: 0.5, Test Acc: 0.49960318207740784
Epoch [4/50], Loss: 0.7169, Train Acc: 0.5916666388511658, Test Acc: 0.5039682388305664
Epoch [5/50], Loss: 0.6868, Train Acc: 0.5666666626930237, Test Acc: 0.5051587224006653
New best acc: 0.5178571343421936 epoch 5
Epoch [6/50], Loss: 0.6683, Train Acc: 0.5611110925674438, Test Acc: 0.5178571343421936
Epoch [7/50], Loss: 0.6395, Train Acc: 0.6083333492279053, Test Acc: 0.5089285969734192
Epoch [8/50], Loss: 0.6327, Train Acc: 0.6361111402511597, Test Acc: 0.5172619223594666
New best acc: 0.5281745791435242 epoch 8
Epoch [9/50], Loss: 0.6245, Train Acc: 0.625, Test Acc: 0

[I 2024-06-04 00:37:24,140] Trial 1 finished with value: 0.5646825432777405 and parameters: {'batch_size': 1422, 'lr': 0.0010608231272589844}. Best is trial 1 with value: 0.5646825432777405.


...

New best acc: 0.5091269612312317 epoch 0
Epoch [1/50], Loss: 0.8448, Train Acc: 0.5055555701255798, Test Acc: 0.5091269612312317
Epoch [2/50], Loss: 0.7629, Train Acc: 0.5611110925674438, Test Acc: 0.5009920597076416
Epoch [3/50], Loss: 0.7187, Train Acc: 0.5888888835906982, Test Acc: 0.5073412656784058
New best acc: 0.5130952596664429 epoch 3
Epoch [4/50], Loss: 0.6954, Train Acc: 0.5972222089767456, Test Acc: 0.5130952596664429
New best acc: 0.521230161190033 epoch 4
Epoch [5/50], Loss: 0.6591, Train Acc: 0.6027777791023254, Test Acc: 0.521230161190033
New best acc: 0.5295634865760803 epoch 5
Epoch [6/50], Loss: 0.6361, Train Acc: 0.6000000238418579, Test Acc: 0.5295634865760803
Epoch [7/50], Loss: 0.6128, Train Acc: 0.6583333611488342, Test Acc: 0.5134920477867126
Epoch [8/50], Loss: 0.5944, Train Acc: 0.6166666746139526, Test Acc: 0.5224206447601318
New best acc: 0.5378968119621277 epoch 8
Epoch [9/50], Loss: 0.5886, Train Acc: 0.6861110925674438, Test Acc: 0.5378968119621277
Epoch

[I 2024-06-04 00:39:31,165] Trial 18 finished with value: 0.5623015761375427 and parameters: {'batch_size': 636, 'lr': 0.0017073184922181198}. Best is trial 14 with value: 0.5692460536956787.


Epoch [50/50], Loss: 0.4193, Train Acc: 0.9944444298744202, Test Acc: 0.5519841313362122
New best acc: 0.49940475821495056 epoch 0
Epoch [1/50], Loss: 0.8423, Train Acc: 0.5638889074325562, Test Acc: 0.49940475821495056
New best acc: 0.5013889074325562 epoch 1
Epoch [2/50], Loss: 0.8234, Train Acc: 0.5027777552604675, Test Acc: 0.5013889074325562
Epoch [3/50], Loss: 0.8058, Train Acc: 0.49444442987442017, Test Acc: 0.48948413133621216
Epoch [4/50], Loss: 0.7886, Train Acc: 0.5083333253860474, Test Acc: 0.49761903285980225
New best acc: 0.5146825313568115 epoch 4
Epoch [5/50], Loss: 0.7826, Train Acc: 0.5166666507720947, Test Acc: 0.5146825313568115
Epoch [6/50], Loss: 0.7751, Train Acc: 0.5361111164093018, Test Acc: 0.5089285969734192
Epoch [7/50], Loss: 0.7617, Train Acc: 0.519444465637207, Test Acc: 0.502579391002655
Epoch [8/50], Loss: 0.7570, Train Acc: 0.5583333373069763, Test Acc: 0.4968253970146179
Epoch [9/50], Loss: 0.7400, Train Acc: 0.5305555462837219, Test Acc: 0.5065476298

[I 2024-06-04 00:39:41,235] Trial 19 finished with value: 0.5349206328392029 and parameters: {'batch_size': 1469, 'lr': 0.00020622740903339162}. Best is trial 14 with value: 0.5692460536956787.


Epoch [50/50], Loss: 0.5802, Train Acc: 0.6388888955116272, Test Acc: 0.5198412537574768
{'batch_size': 1738, 'lr': 0.0016801991703897096}


In [41]:
accs = []
for excluded_human in range(15):
    data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
    data, labels, groups = flatten_data(data, labels, groups)
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    data, labels, groups = select_emotions(data, labels, groups, 0, 2)
    indices = groups != excluded_human
    data = data[indices]
    labels = labels[indices]
    groups = groups[indices]

    for human_test in range(15):
        if human_test == excluded_human:
            continue
        train_data, train_labels, test_data, test_labels = train_test_split(data, labels, groups, human_test)
        # Split data into training and testing sets

        # Convert to tensors and create dataloaders
        batch_size_train = 1024
        batch_size_test = 1738

        train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
        train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)[..., None]
        train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

        test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
        test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)[..., None]
        test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

        # Parameters for the autoencoder
        lr = 0.0016801991703897096
        input_dim = data.shape[1]
        encoding_dim = 128  # Set the desired encoding dimension

        # Initialize the autoencoder
        autoencoder = Autoencoder(input_dim, encoding_dim)

        # Initialize weights of the autoencoder
        def init_weights(m):
            if type(m) == torch.nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)
        autoencoder.apply(init_weights)

        # Loss function and optimizer
        criterion_enc = torch.nn.MSELoss()
        criterion_cls = torch.nn.BCELoss()
        optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

        # Training loop
        num_epochs = 50
        best_acc = 0
        for epoch in range(num_epochs):
            for data_batch, labels_batch in dataloader:
                optimizer.zero_grad()
                reconstructed, cls = autoencoder(data_batch)
                loss1 = criterion_cls(cls, labels_batch) * 0.1
                loss2 = criterion_enc(reconstructed, data_batch)
                loss = loss1 + loss2
                loss.backward()
                optimizer.step()
            
            with torch.no_grad():
                reconstructed, cls = autoencoder(data_tensor)
                cls = cls > 0.5
                train_acc = torch.sum(cls == data_labels) / data_labels.shape[0]
                # test acc

                reconstructed, cls = autoencoder(test_data_tensor)
                cls = cls > 0.5
                test_acc = torch.sum(cls == test_labels_tensor) / test_labels.shape[0]
                if epoch > 6 and test_acc < 0.5:
                    break
                if test_acc > best_acc:
                    best_acc = test_acc
                    torch.save(autoencoder.state_dict(), f"models/best_autoencoder_{excluded_human}_{human_test}.pt")
                    # print(f"New best acc: {best_acc} epoch {epoch}")
            # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Acc: {train_acc.item()}, Test Acc: {test_acc.item()}')
        print(f"Human {human_test} best acc: {best_acc}")
        accs.append(best_acc)
    print(f"Mean acc: {np.mean(accs)}")


Human 1 best acc: 0.894444465637207
Human 2 best acc: 0.8861111402511597
Human 3 best acc: 0.9111111164093018
Human 4 best acc: 0.7749999761581421
Human 5 best acc: 0.8583333492279053
Human 6 best acc: 0.9416666626930237
Human 7 best acc: 0.5805555582046509
Human 8 best acc: 0.5722222328186035
Human 9 best acc: 0.8416666388511658
Human 10 best acc: 0.8833333253860474
Human 11 best acc: 0.8666666746139526
Human 12 best acc: 0.9194444417953491
Human 13 best acc: 0.8222222328186035
Human 14 best acc: 0.5555555820465088
Mean acc: 0.8077380061149597
Human 0 best acc: 0.9166666865348816
Human 2 best acc: 0.9111111164093018
Human 3 best acc: 0.8999999761581421
Human 4 best acc: 0.7666666507720947
Human 5 best acc: 0.8444444537162781
Human 6 best acc: 0.9305555820465088
Human 7 best acc: 0.8777777552604675
Human 8 best acc: 0.8500000238418579
Human 9 best acc: 0.8805555701255798
Human 10 best acc: 0.8888888955116272
Human 11 best acc: 0.8444444537162781
Human 12 best acc: 0.9333333373069763
Hu

In [42]:
for em1 in range(4):
    for em2 in range(em1+1, 4):
        if em1 == 0 and em2 == 2:
            continue
        for excluded_human in range(15):
            accs = []
            data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
            data, labels, groups = flatten_data(data, labels, groups)
            scaler = StandardScaler()
            data = scaler.fit_transform(data)
            data, labels, groups = select_emotions(data, labels, groups, em1, em2)
            indices = groups != excluded_human
            data = data[indices]
            labels = labels[indices]
            groups = groups[indices]

            for human_test in range(15):
                if human_test == excluded_human:
                    continue
                train_data, train_labels, test_data, test_labels = train_test_split(data, labels, groups, human_test)
                # Split data into training and testing sets

                # Convert to tensors and create dataloaders
                batch_size_train = 1024
                batch_size_test = 1738

                train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
                train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)[..., None]
                train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
                train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

                test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
                test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)[..., None]
                test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
                test_dataloader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

                # Parameters for the autoencoder
                lr = 0.0016801991703897096
                input_dim = data.shape[1]
                encoding_dim = 128  # Set the desired encoding dimension

                # Initialize the autoencoder
                autoencoder = Autoencoder(input_dim, encoding_dim)

                # Initialize weights of the autoencoder
                def init_weights(m):
                    if type(m) == torch.nn.Linear:
                        torch.nn.init.xavier_uniform_(m.weight)
                        m.bias.data.fill_(0.01)
                autoencoder.apply(init_weights)

                # Loss function and optimizer
                criterion_enc = torch.nn.MSELoss()
                criterion_cls = torch.nn.BCELoss()
                optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

                # Training loop
                num_epochs = 50
                best_acc = 0
                for epoch in range(num_epochs):
                    for data_batch, labels_batch in dataloader:
                        optimizer.zero_grad()
                        reconstructed, cls = autoencoder(data_batch)
                        loss1 = criterion_cls(cls, labels_batch) * 0.1
                        loss2 = criterion_enc(reconstructed, data_batch)
                        loss = loss1 + loss2
                        loss.backward()
                        optimizer.step()
                    
                    with torch.no_grad():
                        reconstructed, cls = autoencoder(data_tensor)
                        cls = cls > 0.5
                        train_acc = torch.sum(cls == data_labels) / data_labels.shape[0]
                        # test acc

                        reconstructed, cls = autoencoder(test_data_tensor)
                        cls = cls > 0.5
                        test_acc = torch.sum(cls == test_labels_tensor) / test_labels.shape[0]
                        if epoch > 6 and test_acc < 0.5:
                            break
                        if test_acc > best_acc:
                            best_acc = test_acc
                            torch.save(autoencoder.state_dict(), f"models/best_autoencoder_{em1}_{em2}_{excluded_human}_{human_test}.pt")
                            # print(f"New best acc: {best_acc} epoch {epoch}")
                    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Acc: {train_acc.item()}, Test Acc: {test_acc.item()}')
                print(f"Human {human_test} best acc: {best_acc}")
                accs.append(best_acc)
            print(f"Mean acc: {np.mean(accs)}")

Human 1 best acc: 0.7277777791023254
Human 2 best acc: 0.7333333492279053
Human 3 best acc: 0.7055555582046509
Human 4 best acc: 0.5472221970558167
Human 5 best acc: 0.7611111402511597
Human 6 best acc: 0.75
Human 7 best acc: 0.5416666865348816
Human 8 best acc: 0.5333333611488342
Human 9 best acc: 0.7416666746139526
Human 10 best acc: 0.574999988079071
Human 11 best acc: 0.6944444179534912
Human 12 best acc: 0.5277777910232544
Human 13 best acc: 0.5472221970558167
Human 14 best acc: 0.6166666746139526
Mean acc: 0.6430555582046509
Human 0 best acc: 0.7250000238418579
Human 2 best acc: 0.7194444537162781
Human 3 best acc: 0.7277777791023254
Human 4 best acc: 0.5388888716697693
Human 5 best acc: 0.7388888597488403
Human 6 best acc: 0.7527777552604675
Human 7 best acc: 0.5388888716697693
Human 8 best acc: 0.5583333373069763
Human 9 best acc: 0.7222222089767456
Human 10 best acc: 0.5583333373069763
Human 11 best acc: 0.699999988079071
Human 12 best acc: 0.5333333611488342
Human 13 best acc

In [23]:
data, labels, groups = load_dataset("all_features_4sec_epoch.npy")
data, labels, groups = flatten_data(data, labels, groups)

encdoed_data  = autoencoder.encode(torch.tensor(data, dtype=torch.float32)).detach().numpy()
print(encdoed_data.shape)

encoded_data = encdoed_data
encoded_data, labels, groups = select_emotions(encoded_data, labels, groups, 0, 2)
train_data, train_labels, test_data, test_labels = train_test_split(encoded_data, labels, groups, 0)

print(f"train_data: {train_data.shape}, test_data: {test_data.shape}")


(10800, 128)
train_data: (5040, 128), test_data: (360, 128)


In [25]:
accs = []
for human_test in range(15):
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(train_data, train_labels)
    pred = knn.predict(test_data)
    acc = accuracy_score(test_labels, pred)
    accs.append(acc)
print(f" {np.mean(accs)}")

 0.5083333333333331


In [ ]:
train_data, train_labels, test_data, test_labels = train_test(data, labels, test_subjects=[i])

train_mask = np.logical_or((train_labels == 0), (train_labels == 2))
test_mask = np.logical_or((test_labels == 0), (test_labels == 2))

train_labels = train_labels[train_mask]
train_data = train_data[train_mask]

test_labels = test_labels[test_mask]
test_data = test_data[test_mask]

test_data = np.reshape(test_data, (test_data.shape[0], -1))
train_data = np.reshape(train_data, (train_data.shape[0], -1))

train_labels[train_labels == 0] = 0
test_labels[test_labels == 0] = 0

test_labels[test_labels == 2] = 1
train_labels[train_labels == 2] = 1


hidden_size = 500
net = nn.Sequential(
    nn.Linear(10*62*4, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
).to('cuda')

optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = nn.BCELoss()

train_data = torch.tensor(train_data, dtype=torch.float32).to('cuda')
train_labels = torch.tensor(train_labels, dtype=torch.long).to('cuda')
test_data = torch.tensor(test_data, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

from torch.utils.data import DataLoader, TensorDataset

# Create TensorDatasets for training and testing data
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# Create DataLoaders for training and testing data
train_loader = DataLoader(train_dataset, batch_size=5000, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# convert labels to one-hot encoding
# train_labels = F.one_hot(train_labels, num_classes=4).float()
# test_labels = F.one_hot(test_labels, num_classes=4).float()



In [ ]:
net.train()
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
net.apply(init_weights)
model = net
for epoch in range(100000):
    net.train()
    # clear cuda cache
    torch.cuda.empty_cache()
    optimizer.zero_grad()
    out = net(train_data)
    # print(f"out: {out.shape}, train_labels: {train_labels.shape}, train_data: {train_data.shape}")
    loss = loss_fn(out[..., 0], train_labels.float())
    
    loss.backward()
    optimizer.step()
            # Training loop with DataLoader
    # for batch_data, batch_labels in train_loader:
    #     batch_data, batch_labels = batch_data.to('cuda'), batch_labels.to('cuda')
    #     optimizer.zero_grad()
    #     out = model(batch_data)
    #     loss = loss_fn(out[..., 0], batch_labels.float())
    #     optimizer.step()
        
    #     loss.backward()

    if epoch % 10 == 0:
        out = out[..., 0].detach().cpu() > 0.5
        out = out
        acc = accuracy_score(train_labels.cpu(), out)
        # acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
        # test the model
        with torch.no_grad():
            out = net(test_data.to('cuda'))
            # predicted_labels = torch.argmax(out, dim=1)
            # test_non_one_hot = torch.argmax(test_labels, dim=1)
            test_acc = accuracy_score(test_labels, out[..., 0].cpu() > 0.5)
        print(f'Epoch {epoch}, loss {loss.item()} train_accuracy {acc} test_accuracy {test_acc}')
        with torch.no_grad():
            # Calculate training accuracy
            # all_train_out = []
            # for batch_data, _ in train_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = model(batch_data)
            #     all_train_out.append(out[..., 0].cpu())
            # all_train_out = torch.cat(all_train_out)
            # train_acc = accuracy_score(train_labels.cpu(), all_train_out > 0.5)
            
            # # Calculate validation accuracy
            # all_test_out = []
            # for batch_data, _ in test_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = net(batch_data)
            #     all_test_out.append(out[..., 0].cpu())
            # all_test_out = torch.cat(all_test_out)
            # val_acc = accuracy_score(test_labels, all_test_out > 0.5)
            
            out = out[..., 0].detach().cpu() > 0.5
            out = out
            train_acc = accuracy_score(train_labels.cpu(), out)
            # acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
            # test the model
            with torch.no_grad():
                out = net(test_data.to('cuda'))
                # predicted_labels = torch.argmax(out, dim=1)
                # test_non_one_hot = torch.argmax(test_labels, dim=1)
                val_acc = accuracy_score(test_labels, out[..., 0].cpu() > 0.5)


        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f'New best val accuracy: {best_val_acc} at epoch {epoch}')
        
        if epoch % 50 == 0:
            print(f'Epoch {epoch}, loss {loss.item()}, train_acc {train_acc}, val_accuracy {val_acc}')
    




/tmp/ipykernel_17444/537343080.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Epoch 0, loss 0.8660377264022827 train_accuracy 0.5043650793650793 test_accuracy 0.5


ValueError: Found input variables with inconsistent numbers of samples: [10080, 720]

In [ ]:
# # generate confusion matrix
# net.eval()
# with torch.no_grad():
#     out = net(test_data.to('cuda'))
#     predicted_labels = torch.argmax(out, dim=1)
#     test_non_one_hot = torch.argmax(test_labels, dim=1)
#     test_acc = accuracy_score(test_non_one_hot, predicted_labels.cpu())
#     cm = confusion_matrix(test_non_one_hot, predicted_labels.cpu())
#     print(f"Test accuracy: {test_acc}")
#     print(cm)
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['0','1','2','3'])
#     disp.plot()
#     plt.show()

In [ ]:
def create_model(trial):
    hidden_size = trial.suggest_int('hidden_size', 100, 2000)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    
    # model = nn.Sequential(
    #     nn.Linear(10*62*4, hidden_size*2),
    #     nn.BatchNorm1d(hidden_size*2),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size*2, hidden_size),
    #     nn.BatchNorm1d(hidden_size),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size, hidden_size//2),
    #     nn.BatchNorm1d(hidden_size // 2),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size//2, 1),
    #     nn.Sigmoid()
    # ).to('cuda')

    model = nn.Sequential(
        nn.Linear(10*62*4, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_size, 1),
        nn.Sigmoid()
    ).to('cuda')
    model.apply(init_weights)


    optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=weight_decay)

    
    # optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    return model, optimizer

In [ ]:
def objective(trial):
    model, optimizer = create_model(trial)
    loss_fn = nn.BCELoss()
    best_val_acc = 0.0
    for epoch in range(500):    
        model.train()
        torch.cuda.empty_cache()
        
        # Training loop with DataLoader
        # for batch_data, batch_labels in train_loader:
        #     batch_data, batch_labels = batch_data.to('cuda'), batch_labels.to('cuda')
        #     optimizer.zero_grad()
        #     out = model(batch_data)
        #     loss = loss_fn(out[..., 0], batch_labels.float())
        #     optimizer.step()
            
        #     loss.backward()

        optimizer.zero_grad()
        out = model(train_data)
        # print(f"out: {out.shape}, train_labels: {train_labels.shape}, train_data: {train_data.shape}")
        loss = loss_fn(out[..., 0], train_labels.float())
        
        loss.backward()
        optimizer.step()

        
        # Validation step
        model.eval()
        with torch.no_grad():
            # Calculate training accuracy
            # all_train_out = []
            # for batch_data, _ in train_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = model(batch_data)
            #     all_train_out.append(out[..., 0].cpu())
            # all_train_out = torch.cat(all_train_out)
            # train_acc = accuracy_score(train_labels.cpu(), all_train_out > 0.5)
            
            # # Calculate validation accuracy
            # all_test_out = []
            # for batch_data, _ in test_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = net(batch_data)
            #     all_test_out.append(out[..., 0].cpu())
            # all_test_out = torch.cat(all_test_out)
            # val_acc = accuracy_score(test_labels, all_test_out > 0.5)
            
            out = out[..., 0].detach().cpu() > 0.5
            out = out
            train_acc = accuracy_score(train_labels.cpu(), out)
            # acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
            # test the model
            with torch.no_grad():
                out = net(test_data.to('cuda'))
                # predicted_labels = torch.argmax(out, dim=1)
                # test_non_one_hot = torch.argmax(test_labels, dim=1)
                val_acc = accuracy_score(test_labels, out[..., 0].cpu() > 0.5)


        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f'New best val accuracy: {best_val_acc} at epoch {epoch}')
        
        if epoch % 50 == 0:
            print(f'Epoch {epoch}, loss {loss.item()}, train_acc {train_acc}, val_accuracy {val_acc}')
    
    return best_val_acc


In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=1)


print(f'Best hyperparameters: {study.best_params}')


[I 2024-06-03 17:52:11,360] A new study created in memory with name: no-name-3cebeba2-b258-4cd1-a2e6-d30e84b1b15a
/tmp/ipykernel_13991/1746426871.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


New best val accuracy: 0.5222222222222223 at epoch 0
Epoch 0, loss 0.8216129541397095, train_acc 0.5001984126984127, val_accuracy 0.5222222222222223
New best val accuracy: 0.5388888888888889 at epoch 1
New best val accuracy: 0.5444444444444444 at epoch 7
New best val accuracy: 0.55 at epoch 22
New best val accuracy: 0.5583333333333333 at epoch 40
New best val accuracy: 0.5611111111111111 at epoch 46
Epoch 50, loss 0.8271164894104004, train_acc 0.4996031746031746, val_accuracy 0.5333333333333333
New best val accuracy: 0.5638888888888889 at epoch 66
Epoch 100, loss 0.8359272480010986, train_acc 0.49404761904761907, val_accuracy 0.5361111111111111
Epoch 150, loss 0.835940420627594, train_acc 0.5007936507936508, val_accuracy 0.5333333333333333
Epoch 200, loss 0.8274425864219666, train_acc 0.5063492063492063, val_accuracy 0.5361111111111111


[W 2024-06-03 17:52:21,781] Trial 0 failed with parameters: {'hidden_size': 1344, 'dropout_rate': 0.40195477741862107, 'lr': 0.017016176881559823, 'weight_decay': 4.692128365995673e-06} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/drakula/.local/lib/python3.12/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_13991/4244170460.py", line 49, in objective
    out = out[..., 0].detach().cpu() > 0.5
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2024-06-03 17:52:21,782] Trial 0 failed with value None.


KeyboardInterrupt: 